In [1]:
import scipy.io
import numpy as np
import random
import pickle
mat = scipy.io.loadmat('xy_Data_new.mat')
xy_data = np.array(mat.get('xy_data_scaled', []))
np.random.shuffle(xy_data)
xy_data = xy_data.round(4)
x_data = xy_data[:,0:24]
y_data = xy_data[:, 24]
print('Before Reshape')
print(x_data.shape)
print(y_data.shape)


x_data = np.resize(x_data, (22823, 24, 1))
y_data = np.resize(y_data, (22823,))
print('After Reshape')
print(x_data.shape)
print(y_data.shape)
# x_data = np.array(mat.get("x_data", []), ndmin=2)
# y_data = np.array(mat.get("y_data", []), ndmin=1)

# data = [x_data, y_data]

# print('xdata')
# print(x_data[0])
# print('\nydata')
# print(y_data[0])
# print('\ndata')
# print(data[1])


Before Reshape
(22823, 24)
(22823,)
After Reshape
(22823, 24, 1)
(22823,)


In [2]:
pickle_out = open("x_data.pickle", "wb")
pickle.dump(x_data, pickle_out)
pickle_out.close()

pickle_out = open("y_data.pickle", "wb")
pickle.dump(y_data, pickle_out)
pickle_out.close()


In [3]:
x_data = pickle.load(open("x_data.pickle", "rb"))
y_data = pickle.load(open("y_data.pickle", "rb"))

In [4]:
seq_length = 23
batch_size = 100
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Flatten, Dropout, BatchNormalization
from tensorflow.keras.callbacks import TensorBoard
import time


NAME = "{}".format(int(time.time()))
tensorboard = TensorBoard(log_dir='logs/{}'.format(NAME))
# Create and Fit the LSTM Network
model = Sequential()
# model.add(LSTM(32, input_shape=(look_back+1, 1), activation='sigmoid', return_sequences=True))
# model.add(LSTM(32, activation='sigmoid'))
print(x_data.shape)
model.add(LSTM(32, input_shape=(x_data.shape[1:]), return_sequences=True))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(LSTM(32, input_shape=(x_data.shape[1:])))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Dense(1))
lr_schedule = tf.keras.callbacks.LearningRateScheduler(
    lambda epoch: 1e-4 * 10**(epoch / 20) if (1e-4 * 10**(epoch / 20)<1e-2) else 1e-2)
checkpointer = tf.keras.callbacks.ModelCheckpoint(filepath="weights.hdf5", monitor="loss", verbose=1, save_best_only=True, save_weights_only=True)
adam = tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, amsgrad=False)
model.compile(loss="huber_loss",
              optimizer=adam,
              metrics=["mean_squared_error"])
print(model.summary())

print("Starting to fit")
history = model.fit(x_data, y_data, 
                    epochs=150, 
                    batch_size=batch_size, 
                    validation_split = 0.2, 
                    verbose=1, 
                    callbacks=[lr_schedule, checkpointer, tensorboard])
model.load_weights('weights.hdf5')

(22823, 24, 1)
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 24, 32)            4352      
                                                                 
 dropout (Dropout)           (None, 24, 32)            0         
                                                                 
 batch_normalization (BatchN  (None, 24, 32)           128       
 ormalization)                                                   
                                                                 
 lstm_1 (LSTM)               (None, 32)                8320      
                                                                 
 dropout_1 (Dropout)         (None, 32)                0         
                                                                 
 batch_normalization_1 (Batc  (None, 32)               128       
 hNormalization)                         

In [7]:
print('Test Data Evaluation')
print(x_data.shape)
print(y_data.shape)
# print(model.evaluate(x = x_data, y = y_data))

Test Data Evaluation
(22823, 24)
(22823,)


In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs/